In [3]:
import pystac
import pypgstac
import psycopg
from datetime import datetime

In [ ]:
store = "link_to_dataset"

extent = pystac.Extent(
    spatial=pystac.SpatialExtent([[8.45, 45.60, 18.07, 49.83]]),
    temporal=pystac.TemporalExtent([[datetime(2019, 1, 1), datetime(2022, 12, 31)]])
)

collection = pystac.Collection(
    id="clms",
    title="Copernicus Land Monitoring Service",
    description="The Copernicus Land Monitoring Service (CLMS) is a component of the European Union’s Copernicus Earth observation programme that provides systematic information on land cover, land use, and biophysical parameters across Europe and globally. It delivers consistent, long-term datasets derived primarily from satellite imagery to support environmental monitoring, spatial planning, agriculture, forestry, and climate applications. The service produces several product lines, including pan-European, local, and global components, enabling both high-resolution mapping and large-scale assessments.",
    extent=extent)

collection.stac_extensions = [
    "https://stac-extensions.github.io/zarr/v1.1.0/schema.json",
]

In [ ]:
vpp = pystac.Asset(
        href="link_to_dataset",
        media_type="application/vnd.zarr; version=3",
        roles=["data"],
        description="The High Resolution Vegetation Phenology and Productivity (HR-VPP) product suite provides comprehensive information on vegetation status and dynamics.",
        title="Vegetation Phenology and Productivity"
    )
# Zarr extension fields (no dedicated PySTAC helper required)
vpp.extra_fields.update({
    "zarr:zarr_format": 3,
    "zarr:node_type": "group",
    "zarr:zarr_format": 3,
        "zarr:node_type": "group",
        "zarr:group": "VPP",
        "bands": [
            {"name": "AMPL", "description": "Season Amplitude"},
            {"name": "EOSD", "description": "End-of-season Date"},
            {"name": "EOSV", "description": "End-of-season Value"},
            {"name": "LENGTH", "description": "Season Length"},
            {"name": "LSLOPE", "description": "Slope of the Green-up Period"},
            {"name": "MAXV", "description": "Season Maximum Value"},
            {"name": "MINV", "description": "Season Minimum Value"},
            {"name": "QFLAG", "description": "Quality Flag"},
            {"name": "RSLOPE", "description": "Slope of the Green-down Period"},
            {"name": "SOSD", "description": "Start-of-season Date"},
            {"name": "SOSV", "description": "Start-of-season Value"},
            {"name": "SPROD", "description": "Seasonal Productivity"},
            {"name": "TPROD", "description": "Total Productivity"},
        ],
})

collection.add_asset("VPP", vpp)

st = pystac.Asset(
        href="link_to_dataset",
        media_type="application/vnd.zarr; version=3",
        roles=["data"],
        title="Seasonal Trajectories",
        description="Seasonal Trajectories (ST) product is a filtered time series of Plant Phenology Index (PPI) provided yearly on a 10-daily basis at 10m resolution."
    )
# Zarr extension fields (no dedicated PySTAC helper required)
st.extra_fields.update({
    "zarr:zarr_format": 3,
        "zarr:node_type": "group",
        "zarr:group": "ST",
        "bands": [
            {"name": "PPI", "description": "Plant Phenology Index"},
            {"name": "QFLAG", "description": "Quality Flag"},
        ], 
})

collection.add_asset("ST", st)

In [8]:
print(collection.to_dict())

{'type': 'Collection', 'id': 'clms', 'stac_version': '1.1.0', 'description': 'The Copernicus Land Monitoring Service (CLMS) is a component of the European Union’s Copernicus Earth observation programme that provides systematic information on land cover, land use, and biophysical parameters across Europe and globally. It delivers consistent, long-term datasets derived primarily from satellite imagery to support environmental monitoring, spatial planning, agriculture, forestry, and climate applications. The service produces several product lines, including pan-European, local, and global components, enabling both high-resolution mapping and large-scale assessments.', 'links': [], 'stac_extensions': ['https://stac-extensions.github.io/zarr/v1.1.0/schema.json'], 'title': 'Copernicus Land Monitoring Service', 'extent': {'spatial': {'bbox': [[8.45, 45.6, 18.07, 49.83]]}, 'temporal': {'interval': [['2019-01-01T00:00:00Z', '2022-12-31T00:00:00Z']]}}, 'license': 'other', 'assets': {'VPP': {'hre

In [ ]:
import requests

collection_url = "link_to_collections"

# Send a POST request to ingest the STAC Collection
collection_ingest_response = requests.post(collection_url, json=collection.to_dict(), auth=("user", "password"))

# Print the response
print("STAC Collection Ingest Response:", collection_ingest_response.text)

STAC Collection Ingest Response: {"id":"clms","description":"The Copernicus Land Monitoring Service (CLMS) is a component of the European Union’s Copernicus Earth observation programme that provides systematic information on land cover, land use, and biophysical parameters across Europe and globally. It delivers consistent, long-term datasets derived primarily from satellite imagery to support environmental monitoring, spatial planning, agriculture, forestry, and climate applications. The service produces several product lines, including pan-European, local, and global components, enabling both high-resolution mapping and large-scale assessments.","stac_version":"1.1.0","links":[{"rel":"items","type":"application/geo+json","href":"https://dev.stac.eodc.eu/ingestion/v1/collections/clms/items"},{"rel":"parent","type":"application/json","href":"https://dev.stac.eodc.eu/ingestion/v1/"},{"rel":"root","type":"application/json","href":"https://dev.stac.eodc.eu/ingestion/v1/"},{"rel":"self","t